In [2]:
import pandas as pd
from mordecai import Geoparser
import re
import numpy as np

In [9]:
# Get a list of predictions for the unseen documents
y_pred = np.load('data/predictions.npy')
y_pred

array([[0.97231105, 0.02768895],
       [0.98147915, 0.01852085],
       [0.95921204, 0.04078796],
       ...,
       [0.98373336, 0.01626664],
       [0.98585236, 0.01414764],
       [0.94833975, 0.05166025]])

In [7]:
seen_df = pd.read_csv('data/0a_labelled_documents.csv')
unseen_df = pd.read_csv('data/0b_unlabelled_documents.csv')

df = (pd.concat([seen_df,unseen_df])
      .sort_values('id')
      .sample(frac=1, random_state=1)
      .reset_index(drop=True)
     )

seen_index = df[df['seen']==1].index
unseen_index = df[df['seen']==0].index
print(df.shape)
df.head()

(699525, 14)


,id,content,title,PY,wosarticle__de,wosarticle__wc,relevant,seen,1 - Adaptation,1 - Impact,1 - Mitigation,wosarticle__dt,tslug,UT__UT
0,2734050,An Ejector heat pump-boosted District Heating ...,Study of an innovative ejector heat pump-boost...,2013.0,NaN,NaN,0.0,0.0,NaN,NaN,NaN,Article,studyofaninnovativeejectorheatpumpboosteddistr...,WOS:000322051300010
1,4964870,Cervical spine spondylosis is so prevalent in ...,Cervical spinal canal body ratio in normal ind...,2020.0,NaN,NaN,0.0,0.0,NaN,NaN,NaN,Article,cervicalspinalcanalbodyratioinnormalindividual...,2-s2.0-85092231046
2,908214,Considering the effect of ash and moisture on ...,Modified respiratory quotient to evaluate the ...,2018.0,NaN,NaN,0.0,0.0,NaN,NaN,NaN,Article,modifiedrespiratoryquotienttoevaluatetheenviro...,WOS:000427373200007
3,4871627,Recent Al Qaeda threats and related jihadi pro...,The oForest Jihado,2009.0,NaN,NaN,0.0,0.0,NaN,NaN,NaN,Article,theoforestjihado,WOS:000269100200003
4,3736036,This paper aims to improve medical strategies ...,Single step synthesis of glutamic/tartaric aci...,2020.0,NaN,NaN,0.0,0.0,NaN,NaN,NaN,Article,singlestepsynthesisofglutamictartaricacidstabi...,2-s2.0-85081347753


In [12]:
df['prediction'] = df['relevant']
df.loc[unseen_index,"prediction"] = y_pred[:,1]
df.head()

,id,content,title,PY,wosarticle__de,wosarticle__wc,relevant,seen,1 - Adaptation,1 - Impact,1 - Mitigation,wosarticle__dt,tslug,UT__UT,prediction
0,2734050,An Ejector heat pump-boosted District Heating ...,Study of an innovative ejector heat pump-boost...,2013.0,NaN,NaN,0.0,0.0,NaN,NaN,NaN,Article,studyofaninnovativeejectorheatpumpboosteddistr...,WOS:000322051300010,0.027689
1,4964870,Cervical spine spondylosis is so prevalent in ...,Cervical spinal canal body ratio in normal ind...,2020.0,NaN,NaN,0.0,0.0,NaN,NaN,NaN,Article,cervicalspinalcanalbodyratioinnormalindividual...,2-s2.0-85092231046,0.018521
2,908214,Considering the effect of ash and moisture on ...,Modified respiratory quotient to evaluate the ...,2018.0,NaN,NaN,0.0,0.0,NaN,NaN,NaN,Article,modifiedrespiratoryquotienttoevaluatetheenviro...,WOS:000427373200007,0.040788
3,4871627,Recent Al Qaeda threats and related jihadi pro...,The oForest Jihado,2009.0,NaN,NaN,0.0,0.0,NaN,NaN,NaN,Article,theoforestjihado,WOS:000269100200003,0.018275
4,3736036,This paper aims to improve medical strategies ...,Single step synthesis of glutamic/tartaric aci...,2020.0,NaN,NaN,0.0,0.0,NaN,NaN,NaN,Article,singlestepsynthesisofglutamictartaricacidstabi...,2-s2.0-85081347753,0.006805


In [13]:
df = df[df['prediction']>0.33]
print(df.shape)
df.head()

(33838, 15)


,id,content,title,PY,wosarticle__de,wosarticle__wc,relevant,seen,1 - Adaptation,1 - Impact,1 - Mitigation,wosarticle__dt,tslug,UT__UT,prediction
52,4963912,Objective To explore the association between n...,Exploration of the association between meteoro...,2021.0,NaN,NaN,0.0,0.0,NaN,NaN,NaN,Article,explorationoftheassociationbetweenmeteorologic...,2-s2.0-85123003389,0.792216
69,4729030,Background. Acute gastroenteritis is a cause o...,ACUTE GASTROENTERITIS IN 3 COMMUNITY-BASED NUR...,1995.0,NaN,NaN,0.0,0.0,NaN,NaN,NaN,Article,acutegastroenteritisin3communitybasednursinghomes,WOS:A1995RY60500014,0.715006
77,4695222,To study the risk factors for Plasmodium vivax...,Risk factors for Plasmodium vivax gametocyte c...,2004.0,NaN,NaN,0.0,0.0,NaN,NaN,NaN,Article,riskfactorsforplasmodiumvivaxgametocytecarriag...,WOS:000226125300001,0.462889
80,4762134,Background: Despite declining age-adjusted str...,Mortality and Predictors of Death 1 Month and ...,2010.0,NaN,NaN,0.0,0.0,NaN,NaN,NaN,Article,mortalityandpredictorsofdeath1monthand3yearsaf...,WOS:000273552100004,0.553015
99,678134,Weather factors are widely studied for their e...,Statistical Modeling Reveals the Effect of Abs...,2014.0,NaN,NaN,0.0,0.0,NaN,NaN,NaN,Article,statisticalmodelingrevealstheeffectofabsoluteh...,WOS:000337735100017,0.958598


In [20]:
# Either get old data or make a new dataframe
try:
    processed_places = pd.read_csv('data/places.csv')
    df = df[~df['id'].isin(processed_places['doc_id'])]
except:
    processed_places = pd.DataFrame()
    
df.shape

(12904, 17)

In [21]:
# Test the geoparser
geo = Geoparser()
geo.geoparse("I took the tube from Oxford Circus to London Bridge, via Bank")

Models path: /home/galm/software/mordecai-env/lib/python3.8/site-packages/mordecai/models/


[{'word': 'Oxford Circus',
  'spans': [{'start': 21, 'end': 34}],
  'country_predicted': 'GBR',
  'country_conf': 0.96374094,
  'geo': {'admin1': 'England',
   'lat': '51.51517',
   'lon': '-0.14181',
   'country_code3': 'GBR',
   'geonameid': '2640727',
   'place_name': 'Oxford Circus Underground Station',
   'feature_class': 'S',
   'feature_code': 'MTRO'}},
 {'word': 'London Bridge',
  'spans': [{'start': 38, 'end': 51}],
  'country_predicted': 'GBR',
  'country_conf': 0.96374094,
  'geo': {'admin1': 'England',
   'lat': '51.50821',
   'lon': '-0.08763',
   'country_code3': 'GBR',
   'geonameid': '6619889',
   'place_name': 'London Bridge',
   'feature_class': 'S',
   'feature_code': 'BDG'}}]

In [22]:
%%capture 
places = []
geos = []

# Go through the rows of the dataframe
for i, row in df.iterrows():
    
    # Get the text we want to geoparse, join title and abstract, get rid of copyright stuff
    t = str(row['title']) + " " + str(row['content'])
    t = t.split("Copyright (C)")[0] 
    t = re.split("\([C-c]\) [1-2][0-9]{3} Elsevier",t)[0] 
    t = t.split("Published by Elsevier")[0] 
    t = t.split("Copyright. (C)")[0] 
    t = re.split("\. \(C\) [1-2][0-9]{3} ",t)[0] 
    t = re.split("\. \(C\) Copyright",t)[0]   
    
    # geoparse
    gp = geo.geoparse(t)
    
    rplaces = []
    continent = None
    for p in gp:
        try:
            a2 = country_alpha3_to_country_alpha2(p["country_predicted"])
            continent = country_alpha2_to_continent_code(a2)
        except:
            pass
        if "geo" in p:
            try:
                a2 = country_alpha3_to_country_alpha2(p["geo"]["country_code3"])
                continent = country_alpha2_to_continent_code(a2)
            except:
                pass
            for key, value in p["geo"].items():
                p[key] = value
            del p["geo"]
            
        p["doc_id"] = row["id"]
            
        rplaces.append(p)
        places.append(p)
    df.loc[i,"continent"] = continent
    df.loc[i,"places"] = len(rplaces)

In [23]:
# Merge all the data together
combined_place_df = processed_places.append(pd.DataFrame.from_dict(places))
print(combined_place_df.shape)
combined_place_df.to_csv("data/places.csv", index=False)
combined_place_df.tail()

(123488, 13)


,word,spans,country_predicted,country_conf,admin1,lat,lon,country_code3,geonameid,place_name,feature_class,feature_code,doc_id
31645,India,"[{'start': 350, 'end': 355}]",IND,0.655191,NA,22,79,IND,1269750,Republic of India,A,PCLI,2316297
31646,El Nino Southern Oscillation,"[{'start': 402, 'end': 430}]",SLV,0.258393,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2316297
31647,Indian Ocean Dipole,"[{'start': 443, 'end': 462}]",IND,0.361042,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2316297
31648,India,"[{'start': 489, 'end': 494}]",IND,0.655191,NA,22,79,IND,1269750,Republic of India,A,PCLI,2316297
31649,India,"[{'start': 712, 'end': 717}]",IND,0.655191,NA,22,79,IND,1269750,Republic of India,A,PCLI,2316297
